## Youtube Data Analysis

In this notebook, I have a dataset of user comments for youtube videos related to animals or pets. I will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the highest fraction of cat or dog owner users who I can recommend video creators to.

Step 1: Identify cat And dog owners and find the users who are cat and/or dog owners.

Step 2: Build and evaluate classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Apply the best classifier to all the users in the dataset. Estimate the fraction of all users who are cat/dog owners.

Step 4: Extract insights about cat and dog owners through finding topics important to cat and dog owners.

Step 5: Identify creators with pet owners in the audience. Find creators whose video commented by the most cat and/or dog owners. In other words, creators with the highest statistically significant percentages of cat and/or dog owners.

## Step 1: Identify Cat And Dog Owners and find the users who are cat and/or dog owners.

#### 1.1. Data Exploration and Cleaning

In [5]:
df_clean=spark.read.csv("/FileStore/tables/animals_comments.csv",inferSchema=True,header=True)
display(df_clean.head(10))

creator_name,userid,comment
Doug The Pug,87.0,I shared this to my friends and mom the were lol
Doug The Pug,87.0,Super cute 😀🐕🐶
bulletproof,530.0,stop saying get em youre literally dumb . have some common sense or dont own this kind of dog. fucking retarded I swear
Meu Zoológico,670.0,Tenho uma jiboia e um largato
ojatro,1031.0,I wanna see what happened to the pigs after that please
Tingle Triggers,1212.0,Well shit now Im hungry
Hope For Paws - Official Rescue Channel,1806.0,when I saw the end it said to adopt I saw different animal sites I was mad that they separated the cute little pups after being together for a long time
Hope For Paws - Official Rescue Channel,2036.0,Holy crap. That is quite literally the most adorable pup Ive ever seen.
Life Story,2637.0,武器はクエストで貰えるんじゃないんですか？
Brian Barczyk,2698.0,Call the teddy Larry


In [6]:
df_clean.count() 

Out[2]: 5820035

In [7]:
df_clean = df_clean.na.drop(subset=["comment"])
df_clean = df_clean.dropDuplicates()

##### Filter those non-English comments out

In [9]:
import nltk
nltk.download('stopwords')

ENGLISH_STOPWORDS = set(nltk.corpus.stopwords.words('english'))
NON_ENGLISH_STOPWORDS = set(nltk.corpus.stopwords.words()) - ENGLISH_STOPWORDS
 
STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}

def is_english(text):
    text = text.lower()
    words = set(nltk.wordpunct_tokenize(text))
    return len(words & ENGLISH_STOPWORDS) > len(words & NON_ENGLISH_STOPWORDS)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.

In [10]:
# from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, BooleanType, DoubleType 

classify_lang = udf(lambda x: is_english(x), BooleanType())

In [11]:
df_w_lang = df_clean.withColumn('is_english', classify_lang('comment'))

df_only_en = df_w_lang.filter(df_w_lang.is_english==True)
df_only_en = df_only_en.select('creator_name', 'userid', 'comment')
df_only_en.count()

Out[6]: 4274218

In [12]:
display(df_only_en.take(10))

creator_name,userid,comment
Gohan The Husky,190661.0,From Germany! Love u r videos and Gohan so much 😍
Hope For Paws - Official Rescue Channel,1806.0,when I saw the end it said to adopt I saw different animal sites I was mad that they separated the cute little pups after being together for a long time
Brian Barczyk,2698.0,Call the teddy Larry
Taylor Nicole Dean,2432708.0,I am interested in getting a Hedgehog. So it would be so helpful if you could make a more updated video about how to take care of a hedgehog.
The Dodo,1424576.0,Really? You have to steal other peoples content to get views? This is just sad. One link and not even to her YouTube. Also at least have the DECENCY to actually ask if you can use it!
Hope For Paws - Official Rescue Channel,190054.0,Kinda sad that their mother isnt still found
Talking Kitty Cat,2911.0,steve: No wet food for a month!:cats immediately stop fighting:
Robin Seplut,294406.0,Aww! What a cute and fluffy cat❤️
Hope For Paws - Official Rescue Channel,2036.0,Holy crap. That is quite literally the most adorable pup Ive ever seen.
Hope For Paws - Official Rescue Channel,2911.0,Its people like Hope For Paws who truly make the world a better place <3


##### Use Vader Sentiment Analysis to get sentiment score (positive: score>=0.05, neutral: -0.05<score<0.05, negative: score<=-0.05)

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
sentiment = udf(lambda x: sid.polarity_scores(x)['compound'], DoubleType())

df_w_comment_score = df_only_en.withColumn('sentiment_score', sentiment('comment'))
# print(sid.polarity_scores('From Germany! Love u r videos and Gohan so much'))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...

In [15]:
def score_to_class(score):
  if score>=0.05:
    return 'Positive'
  elif -0.05<score<0.05:
    return 'neutral'
  else:
    return 'negative'

sentiment_classify = udf(lambda x: score_to_class(x), StringType())
df_w_comment = df_w_comment_score.withColumn('sentiment', sentiment_classify('sentiment_score'))

In [16]:
display(df_w_comment.head(10))

creator_name,userid,comment,sentiment_score,sentiment
Gohan The Husky,190661.0,From Germany! Love u r videos and Gohan so much 😍,0.6696,Positive
Hope For Paws - Official Rescue Channel,1806.0,when I saw the end it said to adopt I saw different animal sites I was mad that they separated the cute little pups after being together for a long time,0.128,Positive
Brian Barczyk,2698.0,Call the teddy Larry,0.0,neutral
Taylor Nicole Dean,2432708.0,I am interested in getting a Hedgehog. So it would be so helpful if you could make a more updated video about how to take care of a hedgehog.,0.8596,Positive
The Dodo,1424576.0,Really? You have to steal other peoples content to get views? This is just sad. One link and not even to her YouTube. Also at least have the DECENCY to actually ask if you can use it!,-0.7877,negative
Hope For Paws - Official Rescue Channel,190054.0,Kinda sad that their mother isnt still found,-0.4228,negative
Talking Kitty Cat,2911.0,steve: No wet food for a month!:cats immediately stop fighting:,-0.7345,negative
Robin Seplut,294406.0,Aww! What a cute and fluffy cat❤️,0.5093,Positive
Hope For Paws - Official Rescue Channel,2036.0,Holy crap. That is quite literally the most adorable pup Ive ever seen.,0.2247,Positive
Hope For Paws - Official Rescue Channel,2911.0,Its people like Hope For Paws who truly make the world a better place <3,0.9201,Positive


In [17]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# you can user your ways to extract the label

df_classify_user = df_w_comment.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .otherwise(0)))

# (large,small)=df_clean.filter(col('label')==1).randomSplit([0.99, 0.01],seed = 100)
# df_clean = small
df_classify_user.createOrReplaceTempView('iden_owner')

In [18]:
%sql

Select label,count(*)
From iden_owner
Group by 1


label,count(1)
1,39265
0,4234953


#### 1.2 Data preprocessing including numeric vector feature generation and obtain training and test data

In [20]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# countvec = CountVectorizer(inputCol='filtered', outputCol='features')
word2Vec = Word2Vec(inputCol="filtered", outputCol="features",vectorSize=20)

In [21]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, remover, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_classify_user)
dataset = pipelineFit.transform(df_classify_user)

In [22]:
cate_num = dataset.groupBy("label").count()
cate_num.show()

+-----+-------+
label| count|
+-----+-------+
 1| 39265|
 0|4234953|
+-----+-------+

In [23]:
(label0_train,label0_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(label1_train, label1_ex)=dataset.filter(col('label')==0).randomSplit([0.0065, 0.9935],seed = 100)
(label1_test, label1_ex2)=label1_ex.randomSplit([0.003, 0.997],seed = 100)

In [24]:
trainingData = label0_train.union(label1_train)
testData=label0_test.union(label1_test)

In [25]:
# trainingData.write.parquet("dbfs:/Filestore/training.parquet")

In [26]:
display(trainingData.head(10))

creator_name,userid,comment,sentiment_score,label,words,filtered,features
Aarons Animals,2285699.0,Never gonna happen not on my dogs watch you filthy communist felines,0.0,1,"List(never, gonna, happen, not, on, my, dogs, watch, you, filthy, communist, felines)","List(never, gonna, happen, dogs, watch, filthy, communist, felines)","List(1, 20, List(), List(0.2503337822854519, 0.18225614447146654, -0.20774291269481182, -0.03516801632940769, -0.11782620957819745, -0.05884992051869631, 0.17715225648134947, 0.04842398362234235, -0.2797582296188921, -0.1657107884529978, 0.0012020301073789597, 0.04911849740892649, 0.004935234959702939, 0.03571598790585995, -0.043676303466781974, 0.09205120848491788, 0.10440967440081295, 0.15981378080323339, -0.19839006755501032, -0.0180969103530515))"
Alex Knappenberger,517292.0,Kid thinks he is cool: My dog could grab his water bottle and drink it out of there how about your dog? Me: my dog!? Well he could .... Umm....... 0:10 .....yeah um my dog is just a normal dog 😐,0.6189,1,"List(kid, thinks, he, is, cool, my, dog, could, grab, his, water, bottle, and, drink, it, out, of, there, how, about, your, dog, me, my, dog, well, he, could, umm, 0, 10, yeah, um, my, dog, is, just, a, normal, dog)","List(kid, thinks, cool, dog, grab, water, bottle, drink, dog, dog, well, umm, 0, 10, yeah, um, dog, normal, dog)","List(1, 20, List(), List(0.16871092593493428, -0.023978168438923985, -0.0877951397315452, 0.12337067386282509, -0.11963025603051247, 0.0819256623324595, 0.04887496969221454, -0.04836835398485786, 0.07178512686177303, -0.3980460970809585, 0.07406207296605173, -0.09226672684675769, 0.16203258696355316, -0.005787633163364309, 0.10812565488250632, -0.04319395634688829, 0.08781321890848247, 0.25075001544074005, -0.07439040598508559, 0.09059439266198559))"
Brave Wilderness,212789.0,The crab is cuter than my dog and when my mom says wanna take the dog for a walk im like dog where? all I see is a beached whale,0.7003,1,"List(the, crab, is, cuter, than, my, dog, and, when, my, mom, says, wanna, take, the, dog, for, a, walk, im, like, dog, where, all, i, see, is, a, beached, whale)","List(crab, cuter, dog, mom, says, wanna, take, dog, walk, im, like, dog, see, beached, whale)","List(1, 20, List(), List(0.13986005022500952, -0.03907567262649536, -0.13026724041750035, 0.14239789785351603, -0.2743004396829444, 0.029896076533865803, 0.019119252761205036, -0.003630560946961244, -0.08519598341857393, -0.25708630892137685, 0.06175308053692182, -0.038810342798630396, 0.06268145975967249, 0.02200352089324345, 0.06462769278635581, 0.02750047544638316, 0.0023026781777540843, 0.25796371440325555, -0.06836107938239971, 0.24448468585809072))"
Brave Wilderness,1008240.0,Omg it’s so adorable it sounds like my cat,0.7661,1,"List(omg, it, s, so, adorable, it, sounds, like, my, cat)","List(omg, adorable, sounds, like, cat)","List(1, 20, List(), List(0.23596538454294205, -0.1652421295642853, -0.2857926726341248, 0.2929233313770965, 0.15470136544900015, 0.11412937790155411, 0.2674281768500805, -0.1548633225262165, 0.11761091947555542, -0.12404463849961758, 0.21599177354946733, 0.17296529263257981, 0.11921237334609032, 0.13287490345537664, 0.4395999148488045, -0.12394305430352688, -0.09271230548620224, 0.12991311959922314, 0.10959469443187118, 0.47557075321674347))"
Brian Barczyk,124380.0,Arty will get better! I will pray for you guys ! I was so freaked out when my dog started to throw up 🤢 but she is moving and shaking like new now ! Best of luck ! -Kaitlyn 😘❤️🐶❤️😘 ps tell arty that we all love him and Suzee,0.9707,1,"List(arty, will, get, better, i, will, pray, for, you, guys, i, was, so, freaked, out, when, my, dog, started, to, throw, up, but, she, is, moving, and, shaking, like, new, now, best, of, luck, kaitlyn, ps, tell, arty, that, we, all, love, him, and, suzee)","List(arty, get, better, pray, guys, freaked, dog, started, throw, moving, shaking, like, new, best, luck, kaitlyn, ps, tell, arty, love, suzee)","List(1, 20, L

In [27]:
display(trainingData.groupBy("label").count())
# view training data's number of each label

label,count
1,27505
0,27506


In [28]:
display(testData.groupBy("label").count())
# view test data's number of each label

label,count
1,11760
0,12637


## Step 2: Build And tune Classifiers for the cat and dog owners and measure the performance of the classifiers.

#### 2.1 Logistic Regression

In [31]:
from pyspark.ml.classification import GBTClassifier, GBTClassificationModel, LogisticRegression, LogisticRegressionModel, RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

In [32]:
lr = LogisticRegression(maxIter=8, regParam=0.05, elasticNetParam=0.8)
lrModel = lr.fit(trainingData)

print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [0.0,0.0,0.840149054701591,0.36922657483168464,-0.3313616381464569,0.20075403559147442,2.7900831345205983,0.0,-0.683205478816589,-3.447916857513344,0.0,-0.8850310074480583,0.0,0.0,0.01097930210410849,0.4006795614737895,0.0,0.0,0.501666626562054,0.0]
Intercept: -0.4543062777488345

In [33]:
evaluator = BinaryClassificationEvaluator()

print('AUC score for Logistic Regression without tuning the parameter: {}'.format(evaluator.evaluate(lrModel.transform(testData))))

AUC score for Logistic Regression without tuning the parameter: 0.8516301236408144

##### Parameter Tuning and K-fold cross-validation

In [35]:
paramGrid = ParamGridBuilder()\
            .addGrid(lr.regParam,[0.01,0.05,0.1,0.2])\
            .addGrid(lr.maxIter,[8,10,12,14])\
            .build()
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)

cvModel = crossval.fit(trainingData)


In [36]:
best_model = cvModel.bestModel
best_model.extractParamMap()

Out[33]: {Param(parent='LogisticRegression_2b8e7415e410', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_2b8e7415e410', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.8,
 Param(parent='LogisticRegression_2b8e7415e410', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_2b8e7415e410', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_2b8e7415e410', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LogisticRegression_2b8e7415e410', name='labelCol', doc='label column name'): 'label',
 Param(parent='LogisticRegression_2b8e7415e410', name='maxIter', doc='maximum number of iterations (>= 0)'): 10,
 Param(parent='LogisticRegression_2b8e7415e410', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LogisticRegression_2b8e7415e410', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability',
 Param(parent='LogisticRegression_2b8e7415e410', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent='LogisticRegression_2b8e7415e410', name='regParam', doc='regularization parameter (>= 0)'): 0.01,
 Param(parent='LogisticRegression_2b8e7415e410', name='standardization', doc='whether to standardize the training features before fitting the model'): True,
 Param(parent='LogisticRegression_2b8e7415e410', name='threshold', doc='threshold in binary classification prediction, in range [0, 1]'): 0.5,
 Param(parent='LogisticRegression_2b8e7415e410', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0)'): 1e-06}

#### 2.2 Random Forest

In [38]:
trainingData.show(10)

+------------------+---------+--------------------+----------+-----+--------------------+--------------------+--------------------+
 creator_name| userid| comment|is_english|label| words| filtered| features|
+------------------+---------+--------------------+----------+-----+--------------------+--------------------+--------------------+
 Aarons Animals|2285699.0|Never gonna happe...| true| 1|[never, gonna, ha...|[never, gonna, ha...|[0.25033378228545...|
Alex Knappenberger| 517292.0|Kid thinks he is ...| true| 1|[kid, thinks, he,...|[kid, thinks, coo...|[0.16871092593493...|
 Brave Wilderness| 212789.0|The crab is cuter...| true| 1|[the, crab, is, c...|[crab, cuter, dog...|[0.13986005022500...|
 Brave Wilderness|1008240.0|Omg it’s so adora...| true| 1|[omg, it, s, so, ...|[omg, adorable, s...|[0.23596538454294...|
 Brian Barczyk| 124380.0|Arty will get bet...| true| 1|[arty, will, get,...|[arty, get, bette...|[0.11461314811770...|
 Brian Barczyk| 410371.0|that happend to m...| true| 1|[that, happend, t...| [happend, dog]|[0.16299606487154...|
 Brian Barczyk|1766603.0|Im really crying ...| true| 1|[im, really, cryi...|[im, really, cryi...|[0.04498636667781...|
 Brian Barczyk|1996262.0|Number 5 because ...| true| 1|[number, 5, becau...|[number, 5, cats,...|[-0.0641371905138...|
 Brian Barczyk|2351519.0|I feel you my dog...| true| 1|[i, feel, you, my...| [feel, dog, put]|[0.10272719711065...|
 Bully Max| 954499.0|Thats why I adopt...| true| 1|[thats, why, i, a...|[thats, adopted, ...|[0.02936820100460...|
+------------------+---------+--------------------+----------+-----+--------------------+--------------------+--------------------+
only showing top 10 rows

In [39]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features', seed=42)

paramGrid = ParamGridBuilder()\
            .addGrid(rf.numTrees,[64,100])\
            .addGrid(rf.maxDepth,[8,10])\
            .build()
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)
cv_rfModel = crossval.fit(trainingData)


java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:286)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:250)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:245)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:242)
	at com.databricks.rpc.AbstractJettyClient.withAttributionContext(BaseJettyClient.scala:85)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:270)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:186)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:142)
	at com.databricks.rpc.DynamicJettyClient.sendAsync(BaseJettyClient.scala:571)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.send(DriverClient.scala:107)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:455)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:326)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:250)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:245)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:242)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:265)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:326)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [40]:
best_rf_model = cv_rfModel.bestModel
best_rf_model.extractParamMap()
save_rf_path = 'Filestore/model/best_rf_model'
best_rf_model.save(rf_path)

#### 2.3 Gradient boosting

In [42]:
gbt = GBTClassifier(labelCol='label', featuresCol='features', seed=42)

paramGrid = ParamGridBuilder()\
            .addGrid(gbt.stepSize,[0.05,0.1])\
            .addGrid(gbt.maxIter,[8,10])\
            .addGrid(gbt.maxDepth,[5,6])\
            .build()
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)
cv_gbtModel = crossval.fit(trainingData)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2623140955701084> in <module> 
 15 numFolds = 5 , 
 16 seed = 42)
 ---> 17 cv_gbtModel = crossval . fit ( trainingData ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /usr/lib/python3.7/multiprocessing/pool.py in next (self, timeout) 
 746 if success : 
 747 return value
 --> 748 raise value
 749 
 750 __next__ = next # XXX 

 /usr/lib/python3.7/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 119 job , i , func , args , kwds = task
 120 try : 
 --> 121 result = ( True , func ( * args , ** kwds ) ) 
 122 except Exception as e : 
 123 if wrap_exception and func is not _helper_reraises_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in <lambda> (f) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 52 
 53 def singleTask ( ) : 
 ---> 54 index , model = next ( modelIter ) 
 55 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 56 return index , metric , model if collectSubModel else None 

 /databricks/spark/python/pyspark/ml/base.py in __next__ (self) 
 60 raise StopIteration ( "No models remaining." ) 
 61 self . counter += 1 
 ---> 62 return index , self . fitSingleModel ( index ) 
 63 
 64 def next ( self ) : 

 /databricks/spark/python/pyspark/ml/base.py in fitSingleModel (index) 
 104 
 105 def fitSingleModel ( index ) : 
 --> 106 return estimator . fit ( dataset , paramMaps [ index ] ) 
 107 
 108 return _FitMultipleIterator ( fitSingleModel , len ( paramMaps ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 128 elif isinstance ( params , dict ) : 
 129 if params : 
 --> 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 132 return self . _fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{

#### 2.4 Get the best model with best hyper-parameter

##### Load the trained models

In [45]:
lr_path = 'Filestore/model/best_lr_model'
rf_path = 'Filestore/model/best_rf_model'
gbt_path = 'Filestore/model/best_gbt_model'
lr_model = LogisticRegressionModel.load(lr_path)
rf_model = RandomForestClassificationModel.load(rf_path)
gbt_model = GBTClassificationModel.load(gbt_path)

##### Evaluate models with the test dataset

In [47]:
evaluator = BinaryClassificationEvaluator()

rf_predict = rf_model.transform(testData)
print('AUC score for fine-tuned Random Forest model: {}'.format(evaluator.evaluate(rf_predict)))

AUC score for fine-tuned Random Forest model: 0.9240284912730623

In [48]:
lr_predict = lr_model.transform(testData)
print('AUC score for fine-tuned Logistic Regression model: {}'.format(evaluator.evaluate(lr_predict)))

AUC score for fine-tuned Logistic Regression model: 0.8796250543028002

Note: We can see the Logistic Regression model **improves its AUC score from 0.8516 to 0.8796** by tuning the parameter.

In [50]:
gbt_predict = gbt_model.transform(testData)
print('AUC score for fine-tuned Gradient-boosted Tree model: {}'.format(evaluator.evaluate(gbt_predict)))

AUC score for fine-tuned Gradient-boosted Tree model: 0.9135970444203636

In [51]:
display(rf_predict.groupby('prediction').count())

prediction,count
0.0,11626
1.0,12771


## Step 3: Apply the best classifier to all the users in the dataset. Estimate the fraction of all users who are cat/dog owners.

#### 3.1 Classify All The Users

In [54]:
prediction = rf_model.transform(dataset)

In [55]:
display(prediction.groupby('prediction').count())

prediction,count
0.0,3473228
1.0,800990


#### 3.2 Get fraction of pet owner users

In [57]:
prediction.createOrReplaceTempView('predict')

In [58]:
%sql

Select prediction, num/total as proportion
From
(Select prediction, count(*) as num
From predict
Group by 1) p
Cross Join (Select count(*) as total From predict) t


prediction,proportion
0.0,0.8125996381092401
1.0,0.1874003618907599


## Step 4: Extract insights about cat and dog owners through finding topics important to cat and dog owners.

#### 4.1 Use CountVectorizer to extract features instead of word2Vec in order to visualize words in each topic

In [61]:
pet_user = prediction.filter(col('prediction')==1)

In [62]:
pet_user.count()

Out[32]: 800990

##### Use nltk stemmer to get the stem of words

In [64]:
from nltk.stem.porter import *

# Instantiate stemmer object
stemmer = PorterStemmer()

# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

# Create user defined function for stemming with return type Array<String>
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

# Create new column with vectors containing the stemmed tokens 
pet_user = pet_user.withColumn("stem_word", stemmer_udf("filtered"))

In [65]:
text1 = pet_user.select("stem_word").rdd.flatMap(lambda a: a.stem_word).countByValue()
wordfreq = sorted(text1.items(),key=lambda x:x[1],reverse=True)

In [66]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = pet_user.select("stem_word").collect()
stopwords = ['cat','dog','like','love','get','got','one','look','know','want','kitti','puppi','lol','dont','never','much','realli','see','also','peopl','need','thing','think','even']

# words = " ".join([(k + " ")*v for k,v in text1.items()])
words = " ".join([word for row in text for word in row[0]])

wcloud = WordCloud(stopwords=stopwords,max_words=1000, background_color="white").generate(words)

fig,ax0=plt.subplots(nrows=1,figsize=(12,8))
ax0.imshow(wcloud,interpolation='bilinear')

ax0.axis("off")
display(fig)

In [67]:
from pyspark.ml.feature import CountVectorizer

countvec = CountVectorizer(inputCol='stem_word', outputCol='vectors')
count_model = countvec.fit(pet_user)
train = count_model.transform(pet_user)

#### 4.2 extract topics of pet owner users via LDA clustering

In [69]:
from pyspark.ml.clustering import LDA

lda = LDA(maxIter=30,k=20,featuresCol='vectors',seed=2)
lda_model = lda.fit(train)

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:286)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:250)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:245)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:242)
	at com.databricks.rpc.AbstractJettyClient.withAttributionContext(BaseJettyClient.scala:85)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:270)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:186)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:142)
	at com.databricks.rpc.DynamicJettyClient.sendAsync(BaseJettyClient.scala:571)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.send(DriverClient.scala:107)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:455)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:326)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:250)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:245)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:242)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:265)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:326)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [70]:
from pyspark.ml.clustering import LocalLDAModel

path = 'dbfs:/Filestore/model/lda_model'
lda_model = LocalLDAModel.load(path)

In [71]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

vocab = count_model.vocabulary

def trans(x):
  return [vocab[i] for i in x]

idx2word = udf(lambda y: trans(y), ArrayType(StringType()))

In [72]:
topics = lda_model.describeTopics(10).withColumn("terms",idx2word("termIndices"))
display(topics.select("topic","terms","termWeights"))

topic,terms,termWeights
0,"List(otherwis, manag, answer, success, heavi, prepar, highli, nala, disgust, heartbeat)","List(0.002753511239482618, 0.002361180601600332, 0.0023605552017384123, 0.0023599399793519356, 0.0019673764775817084, 0.0018396274859288472, 0.0015692128883641088, 0.0015396320784823022, 0.0013742680942309753, 0.001217344435571216)"
1,"List(loki, bitch, 100, florida, trade, otter, patient, respond, whisker, wobbl)","List(0.010569410356347215, 0.002156719946686762, 0.0019270164657736765, 0.0016627641023094688, 0.0013630404773171357, 0.0011038174574266577, 0.0010795811714814727, 0.0010525294700361012, 9.652333173937885E-4, 9.036942233495148E-4)"
2,"List(move, look, whoever, pitch, mango, shampoo, defect, jim, trend, italian)","List(0.020830716513999745, 0.02025594184659543, 0.0020201292260128034, 0.0011482460330011206, 9.83737719276245E-4, 8.45977331809995E-4, 8.333937944722888E-4, 8.030341715530662E-4, 7.387980444602307E-4, 7.311096724017083E-4)"
3,"List(gener, competit, translat, reinforc, tragic, aspca, overload, fever, esp, lovabl)","List(0.001393537145917369, 0.00112092834400439, 9.53036743811484E-4, 9.376355292220524E-4, 8.695307332582123E-4, 6.66637900947543E-4, 6.315636820609265E-4, 5.317197277654631E-4, 4.820940148643618E-4, 4.392757081611066E-4)"
4,"List(okay, shelter, youtub, breeder, chang, bro, mess, flip, tortois, who)","List(0.009792664545535958, 0.009262383257651326, 0.009181905767381926, 0.004030378055061363, 0.0032189419116253858, 0.002521323368543858, 0.002083706512657255, 0.002023072939436958, 0.0018736393857267516, 0.001852440589014055)"
5,"List(kick, buy, hamster, mention, fall, dog, memphi, click, stripe, maya)","List(0.00794663890182564, 0.0036863176841135494, 0.0034395560217161357, 0.003333626342089096, 0.002351169919305298, 0.002349822534940129, 0.0020978445026755355, 0.001971049295696903, 0.0016369577494271307, 0.00142389603380631)"
6,"List(milk, 4th, count, helmet, nicest, habitat, trim, uncl, irish, smell)","List(0.0032521754655874466, 0.0017736854839057114, 0.0013509824772520804, 0.0013379611305481566, 0.0010237932441303395, 9.156661845748229E-4, 8.459411637663644E-4, 7.544521454520792E-4, 7.327469516524592E-4, 6.724463210777417E-4)"
7,"List(obvious, huge, volunt, although, major, horrifi, asriel, japanes, desper, account)","List(0.0058933776862888365, 0.0024398911857580374, 0.0020811106180241914, 0.0019214063891186985, 0.001089151428205419, 0.0010793469362402237, 0.0010337980012431705, 0.0010282181771935621, 9.647898375048683E-4, 8.333131063778994E-4)"
8,"List(paw, climb, american, chewi, smack, dont, organ, smother, ima, decor)","List(0.007383873176066315, 0.003039706965814243, 0.0023266990748339123, 0.0014448208587298852, 0.0014071988029655837, 0.0012425939791765254, 0.0011919042055856053, 9.830965506288146E-4, 9.481686163019074E-4, 9.342897935757682E-4)"
9,"List(hot, soooo, favourit, haven, resourc, unlik, tht, cycl, suck, graduat)","List(0.004911224181075029, 0.003722172325273909, 0.0013886514418487747, 0.001096660140977006, 0.0010043018066334972, 8.679800321942473E-4, 8.413415849317605E-4, 8.294631776468398E-4, 8.287551414433097E-4, 7.407989793221809E-4)"


In [73]:
from pyspark.sql.functions import explode, arrays_zip

topics_words = topics.withColumn("termWithProb", explode(arrays_zip("terms", "termWeights")))
topics_words.createOrReplaceTempView("topics")


In [74]:
%sql

Select topic, termWithProb["terms"] as term, termWithProb["termWeights"] as probability
From topics

-- get words in each topics with probability

topic,term,probability
0,otherwis,0.002753511239482618
0,manag,0.002361180601600332
0,answer,0.0023605552017384123
0,success,0.0023599399793519356
0,heavi,0.0019673764775817084
0,prepar,0.0018396274859288472
0,highli,0.0015692128883641088
0,nala,0.0015396320784823022
0,disgust,0.0013742680942309753
0,heartbeat,0.001217344435571216


In [75]:
%sql

Select termWithProb["terms"] as term, max(termWithProb["termWeights"]) as probability
From topics
Group by 1
Order by 2 Desc

term,probability
year,0.05207532909005037
happi,0.05115082657054772
dog,0.027379612574285592
like,0.02601632817731021
cat,0.02126995270635798
move,0.020830716513999745
look,0.02025594184659543
love,0.0171459501763329
get,0.016283300367835655
night,0.013744157480372231


## Step 5: Identify creators with pet owners in the audience. Find creators whose video commented by the most cat and/or dog owners. In other words, creators with the highest statistically significant percentages of cat and/or dog owners.

In [77]:
import matplotlib.pyplot as plt

Top 20 creators with most audience of pet owner users

In [79]:

top = spark.sql(
      """Select creator_name, count(*) as pet_owner_num, sum(case when sentiment_score>=0.05 then 1 else 0 end)/count(*) as positive_comment_rate
      From predict
      Where prediction=1
      Group by 1
      Order by 2 Desc, 3 Desc
      limit 10""")

display(top)


creator_name,pet_owner_num,positive_comment_rate
The Dodo,77973,0.5736601131160787
Robin Seplut,45130,0.6307334367383115
Brave Wilderness,41736,0.4754169062679701
Taylor Nicole Dean,39560,0.5164812942366026
Hope For Paws - Official Rescue Channel,34504,0.591670530952933
Gohan The Husky,30567,0.5974744004972683
Vet Ranch,23953,0.567027094727174
Gone to the Snow Dogs,23089,0.5696218978734462
Brian Barczyk,20673,0.6123446040729454
Viktor Larkhill,18529,0.5996006260456581


In [80]:
fig, (ax1, ax2) = plt.subplots(figsize=(10,6), ncols=2, constrained_layout=True)
top_pd = top.toPandas()

xticks = [name for name in top.select("creator_name").collect()]
display(xticks)

ax1.plot('creator_name','pet_owner_num',data=top_pd)
ax1.set_xlabel("creator name")
ax1.set_xticklabels(xticks,rotation=90)
ax1.set_ylabel("number of comment from pet owner users")
ax1.set_title("video creators received most comments from pet owner users", y=1.08)
ax1.legend()

ax2.plot('creator_name','positive_comment_rate',data=categories_p)
ax2.set_xlabel("creator name")
ax2.set_xticklabels(xticks,rotation=90)
ax2.set_ylabel("positive comment rate")
ax2.set_title("positive comment rate of each creator", y=1.08)
ax2.legend()

display(fig.show())

Find video creators received more than 1000 comments from pet owner users with **top positive comment rate**

In [82]:
%sql 

Select creator_name, count(*) as comment_num, sum(case when sentiment_score>=0.05 then 1 else 0 end)/count(*) as positive_comment_rate
From predict
Where prediction=1
Group by creator_name
Having count(*)>=1000
Order by 3 Desc, 2 Desc
limit 20

creator_name,comment_num,positive_comment_rate
Cute Cats Kwazi and Uli,1156,0.8685121107266436
Thor Unleashed,1040,0.8384615384615385
Schnauzer Mom,1255,0.7872509960159363
FROSTY Life,3371,0.7502224859092258
MaxluvsMya,6642,0.7488708220415538
Einstein Parrot,1363,0.7329420396184886
Teresa Bullock,1262,0.7297939778129953
Cockatoo Luck,2142,0.7259570494864612
HammyLux,6491,0.7196117701432753
BrookIvy3,3782,0.7176097303014278


Find video creators received more than 10K comments and have **top rate of (number of comments from pet owner users)/(number of comments from all users)**

In [84]:
%sql

with filter as 
  (Select creator_name, count(*) as num
  From predict
  Group by 1
  Having num>=10000
  ),
  part as 
  (Select pre.creator_name, prediction, count(*) as pet_owner_num
  From predict as pre
  Inner join filter as f on pre.creator_name=f.creator_name
  Group by 1,2
  Order by 1)

Select p.creator_name, pet_owner_num, pet_owner_num/num as fraction
From part as p
Inner join filter as f on p.creator_name=f.creator_name
Where p.prediction = 1
Order by 3 DESC



creator_name,pet_owner_num,fraction
Zak Georges Dog Training rEvolution,11302,0.6745449119665772
Gone to the Snow Dogs,23089,0.5263409852508719
MaxluvsMya,6642,0.5211046602855798
meow meow,6232,0.5137251669277059
Dogumentary TV,5149,0.4992727625327257
Robin Seplut,45130,0.49659437273737606
stacyvlogs,15211,0.4791318864774624
Paws Channel,10343,0.47512517800542053
Kitten Academy,7459,0.46228695382708396
ViralBe,8364,0.44681874031732466


#### 5. Analysis and Future work

## Summary

In this project, I dealt with 5.8 million comments on videos about pet on Youtube to do **semantic analysis** and **recommendation to pet owner users**. 
1. **Filtered non-English comments out** and applied Vader **Sentiment Analysis** to classify the comments to Positive, Neutral and Negative.  
2. **Labelled comments which are from pet owner users** (label 1) by keywords like "my dog", "my cat", "I have a kitty" and extracted around 40,000 label 1 data. Added 40,000 label 0 data and splitted them into train and test dataset.
3. Built pipeline including tokenizer, stopword remover and word2Vec to **transform comments to numeric vector features**.
4. Trained classification models including Logistic Regression, Random Forest and Gradient Boosting and tuned their parameters via cross validation. Compared these classfication models and obtained the best performance model random forest with 93% accuracy. 
5. Applied random forest model to rest data to **classify comments from pet owner users, which were around 18.4% among all**.
6. **Extracted main topic of comments from pet owner users**, the most important words including "cat", "dog", "like", "cute" which didn't contain much useful information. I also tried added those words to stopwords, but still could not find interesting content words. This part needs further exploration.
7. Found video creators with more than 10,000 comments, "Zak Georges Dog Training rEvolution", "MaxluvsMya", "Lennon The Bunny" are top 3 creator with **highest fraction of commented by pet owner users**. Also obtained video creators received more than 1000 comments from pet owner users with **top positive comment rate**. For instance, the top 3 creators 'Cute Cats Kwazi and Uli', 'Thor Unleashed' and 'Schnauzer Mom' received respetively 86.8%, 83.8% and 78.7% Positive comments from pet owner users. These creators can be **recommended to users interested in pets**.

## Future work
1. Identify pet owner users based on more information.
2. Explore other methods to extract significant topics for pet owner users to obtain useful information.